<a href="https://colab.research.google.com/github/AnirudhRaghavan11/NFLDataAnalysis/blob/main/NFL_RPO_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs needed

In [2]:
!pip install nflreadpy
!pip install pandas

In [3]:
import nflreadpy as nfl
import pandas as pd
import numpy as np

In [4]:
#load the play by play
df = nfl.load_pbp()
# Convert the Polars DataFrame to a pandas DataFrame
df_pandas = df.to_pandas()

print(df_pandas.head())


   play_id         game_id old_game_id home_team away_team season_type  week  \
0      1.0  2025_01_ARI_NO  2025090705        NO       ARI         REG     1   
1     40.0  2025_01_ARI_NO  2025090705        NO       ARI         REG     1   
2     63.0  2025_01_ARI_NO  2025090705        NO       ARI         REG     1   
3     85.0  2025_01_ARI_NO  2025090705        NO       ARI         REG     1   
4    115.0  2025_01_ARI_NO  2025090705        NO       ARI         REG     1   

  posteam posteam_type defteam  ... out_of_bounds  home_opening_kickoff  \
0    None         None    None  ...           0.0                   0.0   
1     ARI         away      NO  ...           0.0                   0.0   
2     ARI         away      NO  ...           0.0                   0.0   
3     ARI         away      NO  ...           1.0                   0.0   
4     ARI         away      NO  ...           0.0                   0.0   

     qb_epa  xyac_epa  xyac_mean_yardage  xyac_median_yardage xyac_s

In [5]:
# Load the rpy2 extension if not already loaded
%load_ext rpy2.ipython

In [6]:
# Identify the column that likely contains play types
# We can look at the column names to find a relevant one
play_type_column = None
for col in df_pandas.columns:
    if 'play_type' in col.lower() or 'type' in col.lower() and 'play' in col.lower():
        play_type_column = col
        break

if play_type_column:
    print(f"Assuming the play type is in the '{play_type_column}' column.")
    # Get the unique values in the play type column
    unique_play_types = df_pandas[play_type_column].unique()
    print("\nDifferent types of plays in the data:")
    for play_type in unique_play_types:
        print(play_type)
else:
    print("Could not automatically identify a 'play type' column. You may need to manually inspect the column names.")
    print("Available columns:")
    print(df_pandas.columns.tolist())

Assuming the play type is in the 'play_type' column.

Different types of plays in the data:
None
kickoff
run
pass
punt
no_play
field_goal
extra_point
qb_spike
qb_kneel


In [7]:
#Command to load r in python
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [8]:
%%R
install.packages("nflreadr")
library(nflreadr)
df <- load_pbp(2022:2025)

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/nflreadr_1.5.0.tar.gz'
Content type 'application/x-gzip' length 175825 bytes (171 KB)
downloaded 171 KB


The downloaded source packages are in
	‘/tmp/Rtmp09oXYv/downloaded_packages’


In [9]:
#Here we load the rpo data using R, since it is only available in R
%%R -o pbp
options(repos='https://cloud.r-project.org')
if (!requireNamespace("nflreadr", quietly = TRUE))
install.packages("nflreadr")
library(nflreadr)
pbp <- load_pbp(seasons = 2022:2025)

In [10]:
#head(pbp)
pbp = pd.DataFrame(pbp)
pbp.to_csv('pbp.csv', index=False)

In [11]:
#Here, we are sending the dara back into python
%%R -o pfr_passing
options(repos='https://cloud.r-project.org')

if (!requireNamespace("nflreadr", quietly = TRUE)) {
  install.packages("nflreadr", quiet = TRUE)
}

library(nflreadr)

# ✅ New syntax: load advanced passing stats, season summary level
pfr_passing <- load_pfr_advstats(stat_type = "pass", summary_level = "season", seasons = 2022:2025)


In [12]:
#Now back to python to check our data frame
print(pfr_passing.shape)
pd.set_option("display.max_columns", None)
pfr_passing.head()

(381, 37)


,player,team,pass_attempts,throwaways,spikes,drops,drop_pct,bad_throws,bad_throw_pct,season,pfr_id,pocket_time,times_blitzed,times_hurried,times_hit,times_pressured,pressure_pct,batted_balls,on_tgt_throws,on_tgt_pct,rpo_plays,rpo_yards,rpo_pass_att,rpo_pass_yards,rpo_rush_att,rpo_rush_yards,pa_pass_att,pa_pass_yards,intended_air_yards,intended_air_yards_per_pass_attempt,completed_air_yards,completed_air_yards_per_completion,completed_air_yards_per_pass_attempt,pass_yards_after_catch,pass_yards_after_catch_per_completion,scrambles,scramble_yards_per_attempt
1,Patrick Mahomes,KC,648.0,33.0,0.0,33.0,5.4,95.0,15.4,2022,MahoPa00,2.6,160.0,58.0,56.0,140.0,19.4,8.0,474.0,77.1,91.0,713.0,86.0,692.0,4.0,21.0,143.0,1394.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Justin Herbert,LAC,699.0,25.0,2.0,39.0,5.8,87.0,12.9,2022,HerbJu00,2.5,182.0,50.0,72.0,160.0,21.1,23.0,508.0,75.6,41.0,328.0,41.0,328.0,0.0,0.0,188.0,1435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tom Brady,TB,733.0,22.0,4.0,32.0,4.5,116.0,16.4,2022,BradTo00,2.2,130.0,23.0,55.0,100.0,13.2,11.0,545.0,77.1,23.0,160.0,23.0,160.0,0.0,0.0,123.0,948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kirk Cousins,MIN,643.0,27.0,3.0,25.0,4.1,93.0,15.2,2022,CousKi00,2.5,159.0,36.0,84.0,166.0,23.7,14.0,469.0,76.5,20.0,175.0,19.0,175.0,0.0,0.0,191.0,1283.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Joe Burrow,CIN,606.0,17.0,1.0,34.0,5.8,76.0,12.9,2022,BurrJo01,2.2,110.0,22.0,48.0,111.0,16.5,24.0,465.0,79.1,65.0,432.0,58.0,414.0,2.0,18.0,95.0,619.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = pfr_passing.copy()
df = df.fillna(0)
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

rpo_summary = {
    "Total RPO Plays": df["rpo_plays"].sum(),
    "Total RPO Pass Attempts": df["rpo_pass_att"].sum(),
    "Total RPO Rush Attempts": df["rpo_rush_att"].sum(),
}

rpo_summary["Pass %"] = 100 * rpo_summary["Total RPO Pass Attempts"] / rpo_summary["Total RPO Plays"]
rpo_summary["Run %"] = 100 * rpo_summary["Total RPO Rush Attempts"] / rpo_summary["Total RPO Plays"]

pd.Series(rpo_summary)





,0
Total RPO Plays,6787.000000
Total RPO Pass Attempts,5587.000000
Total RPO Rush Attempts,905.000000
Pass %,82.319140
Run %,13.334316


In [14]:
total_pass_yards = df["rpo_pass_yards"].sum()
total_rush_yards = df["rpo_rush_yards"].sum()

rpo_efficiency = {
    "Avg Yards per RPO Pass Attempt": total_pass_yards / df["rpo_pass_att"].sum(),
    "Avg Yards per RPO Rush Attempt": total_rush_yards / df["rpo_rush_att"].sum(),
    "Total RPO Pass Yards": total_pass_yards,
    "Total RPO Rush Yards": total_rush_yards
}

pd.Series(rpo_efficiency).round(2)


,0
Avg Yards per RPO Pass Attempt,7.76
Avg Yards per RPO Rush Attempt,5.52
Total RPO Pass Yards,43359.00
Total RPO Rush Yards,4992.00


In [15]:
df["rpo_yards_per_play"] = df["rpo_yards"] / df["rpo_plays"].replace(0, np.nan)
df["air_yards_per_att"] = df["intended_air_yards"] / df["pass_attempts"].replace(0, np.nan)
df["yac_per_att"] = df["pass_yards_after_catch"] / df["pass_attempts"].replace(0, np.nan)

league_air_yac = {
    "Avg Air Yards / Pass": df["air_yards_per_att"].mean(),
    "Avg YAC / Pass": df["yac_per_att"].mean(),
    "Avg RPO Yards / Play": df["rpo_yards_per_play"].mean(),
}

pd.Series(league_air_yac).round(2)


,0
Avg Air Yards / Pass,3.69
Avg YAC / Pass,1.62
Avg RPO Yards / Play,6.38


In [16]:
%%R -o pfr_rush
pfr_rush <- load_pfr_advstats(stat_type = "rush", summary_level = "season", seasons = 2022:2025)


In [17]:
#Finding rpo plays, found is_rpo in ftn charting dictionary, but in nflreadr, so we use R again and put into python
%%R -o ftn
options(repos='https://cloud.r-project.org')
if (!requireNamespace("nflreadr", quietly = TRUE)) install.packages("nflreadr", quiet = TRUE)
library(nflreadr)

# Pull multiple seasons if you want; start small first to validate keys
ftn <- load_ftn_charting(seasons = 2022:2025)
# This includes: is_rpo, is_play_action, is_screen_pass, is_motion, etc.

/usr/local/lib/python3.12/dist-packages/rpy2/robjects/vectors.py:1017: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


In [18]:
# 1) pick the right keys from FTN
ftn_game_col = "nflverse_game_id" if "nflverse_game_id" in ftn.columns else "ftn_game_id"
ftn_play_col = "nflverse_play_id" if "nflverse_play_id" in ftn.columns else "ftn_play_id"

missing = [c for c in [ftn_game_col, ftn_play_col] if c not in ftn.columns]
if missing:
    raise KeyError(f"FTN keys not found: {missing}. Available: {list(ftn.columns)}")

# 2) build a slim FTN flags frame and rename keys to match PBP
flag_cols = [
    ftn_game_col, ftn_play_col, "season", "week",
    "is_rpo","is_play_action","is_screen_pass","is_motion",
    "is_trick_play","is_qb_out_of_pocket","is_interception_worthy","is_throw_away"
]
ftn_small = ftn[flag_cols].rename(columns={ftn_game_col: "game_id", ftn_play_col: "play_id"}).copy()

# 3) normalize dtypes on both sides
pbp_keys = pbp.copy()
ftn_small["game_id"] = ftn_small["game_id"].astype(str)
pbp_keys["game_id"]  = pbp_keys["game_id"].astype(str)

ftn_small["play_id"] = pd.to_numeric(ftn_small["play_id"], errors="coerce").astype("Int64")
pbp_keys["play_id"]  = pd.to_numeric(pbp_keys["play_id"],  errors="coerce").astype("Int64")

# 4) dedupe and join
ftn_small = ftn_small.drop_duplicates(subset=["game_id","play_id"])
pbp_ftn = pbp_keys.merge(ftn_small, on=["game_id","play_id"], how="inner")
print("joined shape:", pbp_ftn.shape)

# quick peek
cols_show = [c for c in ["game_id","play_id","desc","play_type","is_rpo","is_play_action","is_screen_pass"] if c in pbp_ftn.columns]
pbp_ftn[cols_show].head(8)


joined shape: (150988, 382)


,game_id,play_id,desc,play_type,is_rpo,is_play_action,is_screen_pass
0,2022_01_BAL_NYJ,68,(14:56) 32-Mi.Carter left end to NYJ 41 for 19...,run,False,False,False
1,2022_01_BAL_NYJ,89,"(14:29) (No Huddle, Shotgun) 19-J.Flacco pass ...",pass,False,False,False
2,2022_01_BAL_NYJ,115,(14:25) (No Huddle) 32-Mi.Carter left end to N...,run,False,False,False
3,2022_01_BAL_NYJ,136,"(14:01) (No Huddle, Shotgun) 19-J.Flacco pass ...",pass,False,False,False
4,2022_01_BAL_NYJ,172,"(13:53) 7-B.Mann punts 45 yards to BAL 19, Cen...",punt,False,False,False
5,2022_01_BAL_NYJ,202,(13:42) 8-L.Jackson pass short right to 7-R.Ba...,pass,False,True,False
6,2022_01_BAL_NYJ,230,"(13:21) (No Huddle, Shotgun) 8-L.Jackson pass ...",pass,True,True,False
7,2022_01_BAL_NYJ,254,(12:41) (Shotgun) 8-L.Jackson right tackle to ...,run,False,False,False


In [19]:
df = pbp_ftn.copy()

# keep only offense plays
off_mask = df["play_type"].isin(["run","pass"])
df = df[off_mask].copy()

# ---- find/repair season & week ----
def pick_col(cols, *cands):
    for c in cands:
        if c in cols:
            return c
    # allow suffixed names from merges
    for c in cols:
        if any(c.startswith(k) for k in cands):
            return c
    return None

season_col = pick_col(df.columns, "season")
week_col   = pick_col(df.columns, "week")

if season_col is None:
    # derive from game_id like "2022_01_BAL_NYJ"
    df["season"] = df["game_id"].astype(str).str.split("_", n=1, expand=True)[0].astype(int)
    season_col = "season"

if week_col is None:
    # derive week as the second token in game_id
    df["week"] = (
        df["game_id"].astype(str)
          .str.split("_", expand=True)[1]
          .str.replace(r"\D", "", regex=True)
          .astype(float)
          .astype("Int64")
    )
    week_col = "week"

# ---- pass/run flags ----
is_pass = (df.get("play_type","")=="pass") | (df.get("pass_attempt",0)==1)
is_run  = (df.get("play_type","")=="run")  | (df.get("rush_attempt",0)==1)

# ---- RPO subsets ----
rpo_all  = df[df["is_rpo"] == True]
rpo_pass = rpo_all[is_pass]
rpo_run  = rpo_all[is_run]

def nzmean(s):
    s = pd.to_numeric(s, errors="coerce")
    return s.dropna().mean() if len(s.dropna()) else np.nan

# ---- coverage + per-season rate ----
coverage = {
    "total_off_plays": len(df),
    "rpo_flagged": int((df["is_rpo"] == True).sum()),
}
coverage["coverage_%"] = round(100 * coverage["rpo_flagged"] / max(coverage["total_off_plays"],1), 2)
print(coverage)

per_season = (
    df[[season_col, "is_rpo"]]
      .groupby(season_col)["is_rpo"]
      .mean().mul(100).round(2)
)
print("RPO % by season (FTN charting subset):")
print(per_season)

# ---- league summary: RPO pass vs run ----
summary = {
    "RPO plays (charting)": len(rpo_all),
    "RPO pass plays": len(rpo_pass),
    "RPO run plays": len(rpo_run),
}
tot = max(summary["RPO plays (charting)"], 1)
summary["RPO pass %"] = 100 * summary["RPO pass plays"] / tot
summary["RPO run %"]  = 100 * summary["RPO run plays"]  / tot

if "yards_gained" in df.columns:
    summary["Avg yards per RPO pass"] = nzmean(rpo_pass["yards_gained"])
    summary["Avg yards per RPO run"]  = nzmean(rpo_run["yards_gained"])
if "epa" in df.columns:
    summary["Avg EPA per RPO pass"] = nzmean(rpo_pass["epa"])
    summary["Avg EPA per RPO run"]  = nzmean(rpo_run["epa"])

print(pd.Series(summary).round(3))

# ---- air + YAC on RPO passes ----
air_col = next((c for c in ["air_yards","intended_air_yards","completed_air_yards"] if c in df.columns), None)
yac_col = next((c for c in ["yards_after_catch","pass_yards_after_catch","yak","yac"] if c in df.columns), None)

rpo_pass_metrics = {}
if air_col:
    rpo_pass_metrics["Avg air yards (RPO pass)"] = nzmean(rpo_pass[air_col])
if yac_col:
    rpo_pass_metrics["Avg YAC (RPO pass)"] = nzmean(rpo_pass[yac_col])
if "complete_pass" in df.columns:
    rpo_pass_metrics["Completion % (RPO pass)"] = 100 * nzmean(rpo_pass["complete_pass"])

print(pd.Series(rpo_pass_metrics).round(3) if rpo_pass_metrics else "No air/YAC cols found.")


{'total_off_plays': 115411, 'rpo_flagged': 5458, 'coverage_%': 4.73}
RPO % by season (FTN charting subset):
season_type
POST    3.86
REG     4.77
Name: is_rpo, dtype: float64
RPO plays (charting)      5458.000
RPO pass plays            1783.000
RPO run plays             3675.000
RPO pass %                  32.668
RPO run %                   67.332
Avg yards per RPO pass       6.066
Avg yards per RPO run        4.364
Avg EPA per RPO pass         0.069
Avg EPA per RPO run         -0.053
dtype: float64
Avg air yards (RPO pass)     2.594
Avg YAC (RPO pass)           6.791
Completion % (RPO pass)     76.220
dtype: float64


/tmp/ipython-input-1534285438.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rpo_pass = rpo_all[is_pass]
/tmp/ipython-input-1534285438.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rpo_run  = rpo_all[is_run]


In [20]:
# columns exist in FTN: is_screen_pass, is_motion, is_play_action, n_defense_box, n_blitzers
def nzmean(s):
    s = pd.to_numeric(s, errors="coerce")
    return s.dropna().mean() if len(s.dropna()) else np.nan

df = pbp_ftn.copy()
have_cols = set(df.columns)

# screen vs non-screen within RPO passes
if "is_screen_pass" in have_cols:
    screen_grp = (
        rpo_pass.groupby("is_screen_pass")
                .agg(plays=("play_id","count"),
                     yds=("yards_gained", nzmean),
                     epa=("epa", nzmean),
                     air=("air_yards", nzmean) if "air_yards" in have_cols else ("air", ("air_yards", nzmean)))
    )
    print("RPO pass — screen vs non-screen")
    print(screen_grp)


RPO pass — screen vs non-screen
                plays       yds       epa       air
is_screen_pass                                     
False            1038  6.903661  0.145507  6.427263
True              745  4.899329 -0.037246 -2.463087


In [21]:
for col in ["n_defense_box","n_blitzers"]:
    if col in df.columns:
        binned = pd.cut(df[col], bins=[-1,5,6,7,8,20], labels=["≤5","6","7","8","9+"])
        tmp = df[df["is_rpo"]==True].copy()
        tmp["bin"] = binned
        out = (tmp.groupby(["bin"])
                  .agg(
                       plays=("play_id","count"),
                       pass_share=("play_type", lambda s: (s=="pass").mean()*100),
                       avg_yards=("yards_gained","mean"),
                       avg_epa=("epa","mean"))
               .sort_index())
        print(f"RPO by {col}")
        print(out)
